In [47]:
import rasterio as rio
from rasterio import plot, mask
import matplotlib
import geopandas as gpd
import numpy as np
import os

In [48]:
INDIR='cache/'
OUTDIR='cache/share/'
POLY='https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_land.zip'

!mkdir -p $OUTDIR

In [25]:
geoms = gpd.read_file(POLY)['geometry']

In [54]:
def clip_and_scale(infile, scale=1):
    inpath = os.path.join(INDIR, infile)
    with rio.open(inpath) as src:
        img, transform = rio.mask.mask(src, geoms, crop=False, all_touched=True, nodata=np.nan)
        if scale != 1:
            img *= scale
        profile = src.profile
        profile['nodata'] = np.nan
        outpath = os.path.join(OUTDIR, 'clip_'+infile)
        with rio.open(outpath, 'w', **profile) as dst:
            dst.write(img)

In [56]:
indicators = ['drydays_pr','q99_pr','annual_pr','gt95f_tasmax','gt85f_tasmin']
scenarios = ['rcp45', 'rcp85']
year = 1985
for indicator in indicators:
    for scenario in scenarios:
        for year in range(1985, 2066, 10):
            if indicator in ['q99_pr','annual_pr']:
                scale = 86400
            else:
                scale = 1
            infile=f"q50-diff-{indicator}_{scenario}_ens_{year}-{year+30}.tif"
            clip_and_scale(infile, scale)

In [58]:
!open cache/share